# Manipulation des données de nuscenes avec panda

###  Nuscenes: https://github.com/nutonomy/nuscenes-devkit
### https://www.nuscenes.org/

# Introduction

Notebook de présentation et de découverte du jeu de données NuScenes. NuScenes est un jeu de données qui sert au développement de véhicules autonomes. C'est une grande base de données (+400 go) qui contient:
 - Des informations intrèseques,informations sur le véhicule, comme la vitesse, l'angle du volant, la pression sur les pédales d'accélération et de frein, .... Ces informations sont accéssible en téléchargeant l'extension CAN Bus.
 - Des informations extrèsequs sur l'envrionnement autour du véhicule, détection de tout les objets présents autour (Voiture, camion, piéton, plot de chantier...) qui possèdent une box et une position comme représentation. Ces informations sont extraites à partir de caméra, radar et lidar présent autour du véhicule. Il s'agit d'informations réelles capturer pendant des sessions dans plusieurs villes.
 - Un ensemble de carte où se déplace tout ces objets, sur ces cartes on peut parcourir les routes connaitres leur type, connaitre la signalisation (stop, feu), ... Je ne me suis pas occupé de cette partie car elle est assez compliqué et ne m'interresse pas pour le stage.

Il y a deux versions pour NuScenes, la version normal (+400go) et il existe une version mini (4go) contenant 10 scènes. 
Il est possilble d'utiliser la version normal sans tout télécharger. Sur le site dans la partie "Trainval", on a un zip  metadata de 400 mo mais il suffit. Si par contre, on veut visualiser des scènes (caméra et/ou lidar) il faudra télécharger les zip de 30 go correspondants à 100 scènes chacun. J'en ai téléchargé 3 personnellement ce qui me donner un accès à 300 scènes.

Attention si vous décidez de charger le jeu complet (qu'importe d'avoir téléchargé le jeu en entier), cela risque de mettre du temps sur une machine pas suffisament puissante (je mets 6/15 min c'est aléatoire sur mon pc portable pour charger), de plus je conseille d'avoir au moins 16 go pour pouvoir l'utiliser sans avoir trop de ralentissement.

Mais si vous ne pouvez pas, ce n'est pas grave, car j'ai enregistré les données que j'utilise sous forme de csv pour ce que cela soit plus rapide.

Il y a de plus, une extension CAN Bus

# Import librairies et chargement de NuScenes

In [3]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# data/sets/nuscenes
# D:\Utilisateurs\Alexandre\Repertoire_D\nuscenes\v1 0-trainval01

#nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes')
#nusc = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')
nusc = NuScenes(version='v1.0-mini', dataroot='G:/repertoire_g/data/sets/nuscenes')
nusc_can = NuScenesCanBus(dataroot='G:/repertoire_g/data/sets/nuscenes')


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.6 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


# Prise en main de l'extension CAN 

Tout d'abord, faisons  un rendu de la scène. `field2token` renvoie le token associé au nom de la scène

In [5]:
scene_name = 'scene-0061'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

In [6]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

On prends la scène 61 et je récupère les informations suivantes (vitesse, angle du volant...) et je les mets dans un dataframe.
`get_messages` renvoie une liste de dictionnaire contenant pour chaque enregistrement (toutes les demi-secondes de la scène) les informations du internes du véhicule.

In [7]:
scene_name = 'scene-0061'
dic_scene = nusc_can.get_messages(scene_name,'vehicle_monitor')
dic_scene[0]

{'available_distance': 120,
 'battery_level': 91,
 'brake': 0,
 'brake_switch': 1,
 'gear_position': 7,
 'left_signal': 0,
 'rear_left_rpm': 275.2617,
 'rear_right_rpm': 276.1791,
 'right_signal': 0,
 'steering': 3.0,
 'steering_speed': 1.8118839761882555e-13,
 'throttle': 0,
 'utime': 1532402928127800,
 'vehicle_speed': 31.44,
 'yaw_rate': 0.6000000000000227}

In [8]:
features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
df_scene = pd.DataFrame.from_dict(dic_scene)[features]
df_scene

,vehicle_speed,steering,throttle,left_signal,right_signal
0,31.44,3.0,0,0,0
1,30.73,3.5,0,1,0
2,29.45,3.6,0,1,0
3,28.09,3.6,0,0,0
4,27.07,3.5,0,0,0
5,26.25,3.6,0,1,0
6,25.35,4.0,0,0,0
7,24.51,4.7,0,0,0
8,23.58,6.5,0,1,0
9,22.69,12.6,0,1,0


# Essayer d'apprendre quand mettre le clignotant 
On va maitenant faire un essai pour prendre en main le dataset, avec seulement la vitesse, l'accélération et l'inclinaison du volant. Je ne pense pas qu'il soit possible de prévoir quand mettre un clignotant car il manque certaines informations (trajectoire notamment) mais cela permettra de manipuler NuScenes avant d'attaquer la suite.

## 1/ Prétraitement des données

Pour récupérer la liste des scènes présentant dans jeu de donnnées on peut récupérer l'attribut scene de l'instance NuScene.Par contre, si on utilise le mini jeu de données, on aura que 10 scènes, j'ai donc crée une fonction pour lister toutes les scènes de 1 à 800.

In [16]:
all_scene = [ s["name"] for s in nusc.scene ]
print(all_scene)

['scene-0061', 'scene-0103', 'scene-0553', 'scene-0655', 'scene-0757', 'scene-0796', 'scene-0916', 'scene-1077', 'scene-1094', 'scene-1100']


In [17]:
# Pour utiliser seulement nusc_can 
# (evite de charger nusc scene qui est plus lourd et ne sert actuellement qu'à prendre la liste des scenes)
def get_list_scene():
    list_scene = []
    for i in range(1,800):
        if i < 10:
            list_scene += ["scene-000"+str(i)]
        elif i < 100:
            list_scene += ["scene-00"+str(i)]
        else:
            list_scene += ["scene-0"+str(i)]
    return list_scene
all_scene = get_list_scene()

In [18]:
# Mets des 1 tout le temps pour le clignotant au lieu d'une alternance par défaut
def fill_signal(df,signal):
    i = 0
    index = df.columns.get_loc(signal)
    while df[signal][i] != 1:
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        df.iat[i,index] = 1
        i += 1
    return df

On créer maintenant une nouvelle colonne à notre dataframe qui contient les valeurs du clignotant 0(rien), 1(clignotant gauche), 2(clignotant droit).

In [19]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique que "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column(df,signal):
    i = 0
    tab = []
    while df[signal][i] != 1:
        tab += [0]
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        if signal == "right_signal":
            tab += [2]
        else:
            tab += [1]
        i += 1
    while i < len(df):
        tab += [0]
        i += 1
    df["signal"] = tab
    return df

On récupère les scènes qui n'ont pas de données dans CAN Bus et qui seront à exclure.

In [20]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
print(blacklist)
print( "%s" in all_scene)

['scene-0161', 'scene-0162', 'scene-0163', 'scene-0164', 'scene-0165', 'scene-0166', 'scene-0167', 'scene-0168', 'scene-0170', 'scene-0171', 'scene-0172', 'scene-0173', 'scene-0174', 'scene-0175', 'scene-0176', 'scene-0309', 'scene-0310', 'scene-0311', 'scene-0312', 'scene-0313', 'scene-0314']
False


In [28]:
tab = []
for s in all_scene[:140]:
    if s not in blacklist and s not in ["scene-0419","scene-0420","scene-0040","scene-0037","scene-0136"
                                       ,"scene-0137"] :
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        if df_scene["left_signal"].any():
            #df_scene = fill_signal(df_scene,"left_signal")
            new_df = add_signal_column(df_scene,"left_signal")
            #print(new_df)
        if df_scene["right_signal"].any():
            #df_scene = fill_signal(df_scene,"right_signal")
            new_df = add_signal_column(df_scene,"right_signal")
            #print("right",new_df)
        tab += [new_df]
    
df_total = pd.concat(tab)
print(len(all_scene))
print(df_total)
print(df_total.describe())
df_total.to_csv("cligno_1707.csv")

799
    vehicle_speed  steering  throttle  left_signal  right_signal  signal
0           14.74     191.9        55            0             0       0
1           14.63     206.4       125            0             0       0
2           15.19     206.3       158            1             0       0
3           16.10     192.3       182            1             0       0
4           17.24     120.1       202            0             0       0
..            ...       ...       ...          ...           ...     ...
33          23.11      14.1        16            1             0       0
34          22.95      15.5        19            0             0       0
35          22.72      16.9        45            0             0       0
36          22.91      16.7        97            1             0       0
37          23.42      11.5       118            1             0       0

[5349 rows x 6 columns]
       vehicle_speed     steering     throttle  left_signal  right_signal  \
count    5349.0000

In [29]:
df_total.corr()

,vehicle_speed,steering,throttle,left_signal,right_signal,signal
vehicle_speed,1.000000,-0.009642,0.088292,-0.068969,-0.179062,-0.312831
steering,-0.009642,1.000000,0.003402,0.301272,-0.275218,-0.229618
throttle,0.088292,0.003402,1.000000,-0.087750,-0.056218,-0.133889
left_signal,-0.068969,0.301272,-0.087750,1.000000,-0.143323,0.027734
right_signal,-0.179062,-0.275218,-0.056218,-0.143323,1.000000,0.615849
signal,-0.312831,-0.229618,-0.133889,0.027734,0.615849,1.000000


## 2/ Apprentissage

In [30]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import RandomForestClassifier
import random

In [32]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
print(len(X)/20)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

267.45


### 2/1/ K plus proches voisins et SVM sur une seule variable signal a 3 valeurs

In [33]:
model = neighbors.KNeighborsClassifier(3)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

model2 = RandomForestClassifier(n_estimators=100,random_state = 42)
model2.fit(X_train,y_train)
model2.score(X_test,y_test)

0.7859813084112149


0.8588785046728972

0.80 avec les 1000 scènes (can bus), c'est pas mal pour un modèle basique, on peut améliorer ça.
0.87 avec une forêt aléatoire, c'est encore mieux.
Mais le score reflète-t-il la vérité? Peut-être que dans la majorité de scènes dans le jeu de test le clignotant n'est pas utilisé et donc le score peut-être boosté? Vérifions ça. 

### 2/2 Vérification du modèle

Matrice de confusion:

In [34]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,model.predict(X_test)))
print(confusion_matrix(y_test,model2.predict(X_test)))


[[610  40  33]
 [ 74  86  12]
 [ 62   8 145]]
[[641  24  18]
 [ 50 118   4]
 [ 48   7 160]]


Résultat de la matrice de confusion sur le jeu de test (170 scènes, 6625 échantillons) avec k voisins (gauche) et une forêt aléatoire (droite):
- [3402,  210,  338]-----[3626  145  179]
- [ 288,  549,   63]--------[169  688   43] 
- [ 385,   61, 1329]------[ 257   50 1468]

K voisins:
Résultats assez mitigés le taux de d'erreur pour le clignotant gauche est de 0.5 et du clignotant droit 0.25.

Forêt aléatoire:
Résultats plutôt bon, taux d'erreur pour le clignotant gauche est de 0.30 et du clignotant droit 0.15.

J'obtiens des résultats quasi-identique en changeant l'attribution des valeurs dans la colonne signal (clignotant gauche = 2 au lieu de 1). 
On peut par ailleurs noter qu'il y a un double des cas où le clignotant droite est activé par rapport au clignotant gauche, cela peut être l'une des raisons qui explique la différence de score.